# Policy
* 이 노트북 파일은 중간 과제를 재현Reproduce하는 증거로 활용.
    * 따라서, 이 노트북에 기재된 코드를 실행했을 때, Kaggle에 제출한 결과와 일치해야함
    * 무작위성 등으로 인하여 결과가 매번 달라지는 경우에는 Random Seed 등을 설정해서 항상 같은 결과가 나오도록 할 것(예. scikit-learn의 **random_state** 등)
* 다음과 같은 정책을 어길 시 0점 처리되니 유의할 것
    * 노트북에 기재된 코드를 전체 실행했을 때 오류 등으로 실행이 되지 않는 경우
    * 노트북에 명시되지 않은 별도의 코드 및 라이브러리를 사용하는 경우
        * 과제 수행에 필요한 외부 라이브러리 설치 명령은 이 노트북 내에 명시할 것(Installing Libraries 참조)
        * 별도의 Python 파일은 사용하지 말 것. 필요하다면 이 노트북 내에서 구현.
    * 중간 과제에서 제공한 데이터 이외의 별도의 데이터를 사용하는 경우
    * 노트북 파일 내에서 훈련된 모델이 아닌 다른 모델을 사용하는 경우
    * Kaggle 제출 결과와 노트북 파일 실행으로 나온 결과가 크게 다를 경우

# Installing Libraries

아래에 중간 과제를 수행하는 데 필요한 라이브러리 들을 설치하는 명령어를 넣을 것
예를 들어,
```shell
%conda install sklearn
```
또는
```shell
%pip install -U sklearn
```
버전이 중요하다면, 버전도 명확하게 명시할 것
```shell
%conda install sklearn==1.4.2
```
또는
```shell
%pip install sklearn==1.4.2
```

In [1]:
# 설치하는 라이브러리 명시
!pip  install -q scikit-learn numpy pandas kaggle xgboost catboost

In [2]:
import pandas as pd
import json
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import balanced_accuracy_score

# Data Load
아래에 중간 과제에 활용할 데이터를 불러오는 부분을 넣을 것

In [3]:
np.random.seed(42)
random.seed(42)

In [4]:
USERNAME = "liebenholz" # username
USERKEY = "" # key
json.dump({'username': USERNAME, 'key': USERKEY}, open('kaggle.json', mode='w'))

In [5]:
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c 2024-knu-ml-team-asmt
!unzip -o -qq 2024-knu-ml-team-asmt.zip

mkdir: /Users/gokyulueau/.kaggle/: File exists
2024-knu-ml-team-asmt.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
test_accel  = pd.read_csv('./test_accel.csv',index_col='time',usecols=['x','y','z','time'])
test_gyro   = pd.read_csv('./test_gyro.csv',index_col='time',usecols=['x','y','z','time'])
train_accel = pd.read_csv('./train_accel.csv',index_col='time',usecols=['x','y','z','time'])
train_gyro  = pd.read_csv('./train_gyro.csv',index_col='time',usecols=['x','y','z','time'])
train_label = pd.read_csv('./train_label.csv',index_col='id')
submmision  = pd.read_csv('./sample_submission.csv',index_col='id')

# Data Preprocessing, Feature Engineering, and Model Building

이곳부터는 데이터 전처리, 특성값 공학, 모델 훈련 등의 코드를 자유롭게 넣을 것

In [7]:
submmision = submmision.sort_index()
test_label = submmision.copy()

In [8]:
test_accel  .columns = ['acc_x','acc_y','acc_z']
test_gyro   .columns = ['gyro_x','gyro_y','gyro_z']
train_accel .columns = ['acc_x','acc_y','acc_z']
train_gyro  .columns = ['gyro_x','gyro_y','gyro_z']

In [9]:
test_accel  .index = pd.to_datetime(test_accel.index, unit='s')
test_gyro   .index = pd.to_datetime(test_gyro.index, unit='s')
train_accel .index = pd.to_datetime(train_accel.index, unit='s')
train_gyro  .index = pd.to_datetime(train_gyro.index, unit='s')
train_label .index = pd.to_datetime(train_label.index, unit='s')
test_label  .index = pd.to_datetime(test_label.index, unit='s')

In [10]:
test_accel  = test_accel .ffill().dropna().resample('10ms').nearest()
test_gyro   = test_gyro  .ffill().dropna().resample('10ms').nearest()
train_accel = train_accel.ffill().dropna()
train_accel = train_accel[~train_accel.index.duplicated()]
train_accel = train_accel.resample('10ms').nearest()
train_gyro  = train_gyro .ffill().dropna()
train_gyro  = train_gyro [~train_gyro.index.duplicated()]
train_gyro  = train_gyro .resample('10ms').nearest()
train_label = train_label.ffill().dropna()

In [11]:
merge_train=pd.concat([train_accel,train_gyro],axis=1)
merge_train = merge_train.ffill().bfill().dropna()
merge_train

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z
time,,,,,,
1970-01-01 04:00:00.000,9.561121,-1.004985,0.528814,-0.108123,0.051771,-0.010614
1970-01-01 04:00:00.010,9.561121,-1.004985,0.528814,-0.108123,0.051771,-0.010614
1970-01-01 04:00:00.020,9.561121,-1.004985,0.528814,-0.099266,0.017104,-0.013821
1970-01-01 04:00:00.030,9.847662,-1.066002,0.537787,-0.085368,0.007483,-0.031078
1970-01-01 04:00:00.040,9.847662,-1.066002,0.537787,-0.085368,0.007483,-0.031078
...,...,...,...,...,...,...
1970-01-03 13:05:47.360,9.534423,1.260379,-1.583258,-0.489751,0.213795,0.263714
1970-01-03 13:05:47.370,9.534423,1.260379,-1.583258,-0.489751,0.213795,0.263714
1970-01-03 13:05:47.380,9.550290,0.978815,-1.506617,-0.594105,0.220150,0.281765


In [12]:
merge_test=pd.concat([test_accel,test_gyro],axis=1)
merge_test = merge_test.ffill().bfill().dropna()
merge_test

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z
time,,,,,,
1970-01-01 02:00:00.010,8.540640,-1.877546,3.954028,-0.303788,-0.013181,0.069500
1970-01-01 02:00:00.020,8.540640,-1.877546,3.954028,-0.303788,-0.013181,0.069500
1970-01-01 02:00:00.030,8.609047,-1.755100,3.835923,-0.296809,-0.009532,0.049011
1970-01-01 02:00:00.040,8.609047,-1.755100,3.835923,-0.281611,-0.004791,0.035724
1970-01-01 02:00:00.050,8.654553,-1.616788,3.658991,-0.281611,-0.004791,0.035724
...,...,...,...,...,...,...
1970-01-03 17:04:52.950,-7.900866,1.541866,2.221819,0.078802,0.045815,0.208305
1970-01-03 17:04:52.960,-7.900866,1.541866,2.221819,0.078802,0.045815,0.208305
1970-01-03 17:04:52.970,-8.063671,1.266533,2.248155,0.150884,0.073304,0.082467


In [13]:
scalers = []
for i in range(6):
  scalers.append(MinMaxScaler())

for a,i in enumerate(merge_train.columns):
  merge_train[i] = scalers[a].fit_transform(np.array(merge_train[i]).reshape(len(merge_train[i]),1))
  merge_test[i] = scalers[a].transform(np.array(merge_test[str(i)]).reshape(len(merge_test[i]),1))

In [14]:
fft_train_sensor = []
N = 128
f_s = 200
for t in train_label.index:
    l = train_label.at[t, 'workout']
    fft=[0 for i in range(6)]
    mag=[0 for i in range(6)]
    max_freq=[0 for i in range(6)]
    avg =[0 for i in range(6)]
    sub = merge_train.loc[t - pd.Timedelta(milliseconds=1):t + pd.Timedelta(seconds=3)]
    for i in range(len(merge_train.columns)):
      fft[i] = np.fft.fft(sub[merge_train.columns[i]], n=N)[1:N//2] / N
    for i in range(len(merge_train.columns)):
      mag[i] = np.abs(fft[i])
    freq = np.fft.fftfreq(N, 1.0 / f_s)[1:N//2]
    for i in range(len(merge_train.columns)):
      max_freq[i] = freq[np.argmax(mag[i])]
    for i in range(len(merge_train.columns)):
      avg[i] =  np.dot(mag[i], freq)
    fft_train_sensor.append(
        (t, l, max_freq[0], max_freq[1], max_freq[2], max_freq[3], max_freq[4], max_freq[5]
         ,avg[0] ,avg[1] ,avg[2] ,avg[3] ,avg[4]  ,avg[5]
         )
    )
fft_train_sensor = pd.DataFrame(fft_train_sensor, columns=['timestamps', 'workout',
        'max_freq_AX', 'max_freq_AY', 'max_freq_AZ', 'max_freq_GA', 'max_freq_GY', 'max_freq_GZ',
        'avg_AX' ,'avg_AY' ,'avg_AZ' ,'avg_GA' ,'avg_GY'  ,'avg_GZ'
        ]).set_index(
    'timestamps'
)
fft_train_sensor

,workout,max_freq_AX,max_freq_AY,max_freq_AZ,max_freq_GA,max_freq_GY,max_freq_GZ,avg_AX,avg_AY,avg_AZ,avg_GA,avg_GY,avg_GZ
timestamps,,,,,,,,,,,,,
1970-01-01 04:00:04,1,4.6875,14.0625,6.2500,1.5625,3.1250,3.1250,2.795538,2.744403,1.806397,1.772350,1.908961,1.262873
1970-01-01 04:00:05,1,3.1250,10.9375,18.7500,7.8125,10.9375,3.1250,4.923022,3.010561,2.785997,2.853603,3.983474,2.277454
1970-01-01 04:00:06,1,3.1250,10.9375,7.8125,6.2500,7.8125,4.6875,3.987620,4.422595,1.762738,2.338351,4.046560,1.608907
1970-01-01 04:00:07,1,4.6875,7.8125,20.3125,6.2500,7.8125,4.6875,5.987810,2.662585,3.586014,2.823753,4.924123,1.407230
1970-01-01 04:00:08,1,4.6875,9.3750,12.5000,9.3750,4.6875,1.5625,6.830991,5.238895,3.914385,4.267517,5.593813,1.958451
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-02 15:03:44,8,1.5625,7.8125,1.5625,3.1250,1.5625,3.1250,1.162970,1.114277,1.477949,1.517150,2.917810,1.142151
1970-01-02 15:03:45,8,4.6875,9.3750,3.1250,6.2500,1.5625,4.6875,2.721155,2.688305,2.109890,3.186190,3.556282,3.459079
1970-01-02 15:03:46,8,3.1250,7.8125,1.5625,1.5625,1.5625,7.8125,1.892333,2.900017,2.011099,2.585022,2.351782,1.941894


In [15]:
fft_test_sensor = []
N = 128
f_s = 200
for t in test_label.index:
    fft=[0 for i in range(6)]
    mag=[0 for i in range(6)]
    max_freq=[0 for i in range(6)]
    avg =[0 for i in range(6)]
    sub = merge_test.loc[t - pd.Timedelta(milliseconds=1):t + pd.Timedelta(seconds=3)]
    for i in range(len(merge_test.columns)):
      fft[i] = np.fft.fft(sub[merge_test.columns[i]], n=N)[1:N//2] / N
    for i in range(len(merge_test.columns)):
      mag[i] = np.abs(fft[i])
    freq = np.fft.fftfreq(N, 1.0 / f_s)[1:N//2]
    for i in range(len(merge_test.columns)):
      max_freq[i] = freq[np.argmax(mag[i])]
    for i in range(len(merge_test.columns)):
      avg[i] =  np.dot(mag[i], freq)
    fft_test_sensor.append(
        (t, max_freq[0], max_freq[1], max_freq[2], max_freq[3], max_freq[4], max_freq[5]
         ,avg[0] ,avg[1] ,avg[2] ,avg[3] ,avg[4]  ,avg[5]
         )
    )
fft_test_sensor = pd.DataFrame(fft_test_sensor, columns=['timestamps',
                'max_freq_AX', 'max_freq_AY', 'max_freq_AZ', 'max_freq_GA', 'max_freq_GY', 'max_freq_GZ',
        'avg_AX' ,'avg_AY' ,'avg_AZ' ,'avg_GA' ,'avg_GY'  ,'avg_GZ'
        ]).set_index(
    'timestamps'
)
fft_test_sensor

,max_freq_AX,max_freq_AY,max_freq_AZ,max_freq_GA,max_freq_GY,max_freq_GZ,avg_AX,avg_AY,avg_AZ,avg_GA,avg_GY,avg_GZ
timestamps,,,,,,,,,,,,
1970-01-01 02:00:04,4.6875,6.2500,4.6875,14.0625,4.6875,12.5000,11.116282,5.508368,5.407201,6.138360,12.694680,4.155052
1970-01-01 02:00:05,4.6875,14.0625,4.6875,12.5000,4.6875,4.6875,9.563429,7.981658,5.977519,5.401293,11.856418,4.434677
1970-01-01 02:00:06,4.6875,14.0625,9.3750,14.0625,4.6875,4.6875,9.770762,9.453498,6.827004,4.617286,9.194552,5.391066
1970-01-01 02:00:07,4.6875,15.6250,9.3750,12.5000,4.6875,4.6875,8.791075,6.342092,8.419642,5.282018,8.546592,2.924593
1970-01-01 02:00:08,4.6875,12.5000,9.3750,12.5000,4.6875,4.6875,9.468840,5.941582,6.497646,5.093675,8.036716,3.338238
...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-03 17:04:44,1.5625,1.5625,7.8125,1.5625,1.5625,1.5625,1.384477,0.767711,1.009277,1.225926,1.527176,1.041299
1970-01-03 17:04:45,4.6875,7.8125,1.5625,3.1250,1.5625,14.0625,2.454239,1.822229,2.891491,1.808334,2.944698,2.156367
1970-01-03 17:04:46,1.5625,9.3750,1.5625,3.1250,4.6875,1.5625,0.266275,0.853706,0.343611,0.692001,1.518491,0.474604


In [16]:
fft_train_sensor.corr()['workout']

workout        1.000000
max_freq_AX   -0.490321
max_freq_AY    0.155931
max_freq_AZ   -0.165186
max_freq_GA    0.132063
max_freq_GY   -0.214416
max_freq_GZ    0.032388
avg_AX         0.045999
avg_AY         0.125009
avg_AZ         0.259410
avg_GA         0.124339
avg_GY         0.235456
avg_GZ         0.103280
Name: workout, dtype: float64

In [17]:
x = fft_train_sensor.copy().drop(['workout'],axis=1)
y = fft_train_sensor.copy()[['workout']]

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

model_lr = LogisticRegression(fit_intercept=False)
model_lr.fit(x_train,y_train)
y_pred = model_lr.predict(x_test)
print("Logistic Regression :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(x_train,y_train)
y_pred = model_dt.predict(x_test)
print("Decision Tree :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

model_svc = SVC(C=1.0, probability=True, kernel='sigmoid', coef0 = 1, gamma=0.2)
model_svc.fit(x_train,y_train)
y_pred = model_lr.predict(x_test)
print("SVM :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

model_rf = RandomForestClassifier(random_state = 42)
model_rf.fit(x_train,y_train)
y_pred=model_rf.predict(x_test)
print("Random Forest :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

model_ab = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=10, random_state=42),
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)
model_ab.fit(x_train,y_train)
y_pred=model_ab.predict(x_test)
print("Ada Boost :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

model_xgb = XGBClassifier(
    objective='multi:softmax',
    n_estimators=100,
    learning_rate=0.1,
    subsample=0.8,
    max_depth=10,
    random_state=42
)
model_xgb.fit(x_train,y_train)
y_pred=model_xgb.predict(x_test)
print("XG Boost :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

ens1 = VotingClassifier(estimators=[('model_rf', model_rf), ('model_ab', model_ab), ('model_xgb',model_xgb)], voting='soft')
ens1.fit(x_train,y_train)
y_pred = ens1.predict(x_test)
print("Soft Ensemble :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")

ens2 = VotingClassifier(estimators=[('model_rf', model_rf), ('model_ab', model_ab), ('model_xgb',model_xgb)], voting='hard')
ens2.fit(x_train,y_train)
y_pred = ens2.predict(x_test)
print("Hard Ensemble :",round(balanced_accuracy_score(y_test,y_pred)*100,6),"%")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-

Logistic Regression : 63.879099 %
Decision Tree : 69.303891 %
SVM : 63.879099 %


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest : 79.909104 %


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Ada Boost : 79.748393 %
XG Boost : 81.694913 %


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Soft Ensemble : 80.969205 %


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Hard Ensemble : 80.951137 %


# Final Model Specification
아래에는 위 전 과정을 거쳐서 최종적으로 선정된 모델을 정의하고 훈련할 것

In [25]:
model_rf = RandomForestClassifier(random_state = 42)
model_rf.fit(x,y)

model_ab = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=10, random_state=42),
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)
model_ab.fit(x,y)

model_xgb = XGBClassifier(
    objective='multi:softmax',
    n_estimators=100,
    learning_rate=0.1,
    subsample=0.8,
    max_depth=10,
    random_state=42
)
model_xgb.fit(x,y)

model = VotingClassifier(estimators=[('model_rf', model_rf), ('model_ab', model_ab), ('model_xgb',model_xgb)], voting='soft')
model.fit(x,y)
y_predict=model.predict(x)
print(balanced_accuracy_score(y, y_predict))
y_predict=model.predict(fft_test_sensor)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label

1.0


# Generate Submission
아래에는 Kaggle에 제출한 결과를 PC에 저장하는 코드를 넣을 것.
노트북 실행 후 아래의 코드를 통해 생성된 결과가 Kaggle에 제출된 결과와 일치해야 함.

In [26]:
SUBMIT = pd.DataFrame({
    'id':submmision.sort_index().index ,
    'label':y_predict
})
SUBMIT.to_csv('./submission.csv', index=False)

In [27]:
!kaggle competitions submit -c 2024-knu-ml-team-asmt -f submission.csv -m "Improved Ensemble Soft Learning "

100%|█████████████████████████████████████████| 119k/119k [00:01<00:00, 108kB/s]
Successfully submitted to Team Assignment